In [1]:
import PollingSystem as ps
import StaticRP as srp
import MarkovianRP as mrp
import dynamic_rp as drp

import numpy as np

In [2]:
#set up a simple, symmetric polling system
sys_traffic = 0.1
beta = 1
lambdas = np.array([1/3, 1/3, 1/3])*sys_traffic/beta
sym_sys = ps.PollingSystem(lambdas, beta)
S = np.array([[0, 3, 3],
              [3, 0, 3],
              [3, 3, 0]])

In [3]:
pi, w_rnd = sym_sys.calc_optimal_rp(S, rp_type = 0)
print(pi, w_rnd)
static_rand = mrp.RandomRP(pi)
P, w_mrkv = sym_sys.calc_optimal_rp(S, rp_type = 1)
print(P, w_mrkv)
static_markov = mrp.MarkovianRP(P)

[0.33333333 0.33333333 0.33333333] 6.000000000000001


/home/win/.local/lib/python3.8/site-packages/scipy/optimize/optimize.py:282: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  warnings.warn("Values in x were outside bounds during a "


[[0.00000000e+00 9.99385772e-01 6.14228413e-04]
 [6.72262528e-04 0.00000000e+00 9.99327737e-01]
 [9.99357898e-01 6.42102422e-04 0.00000000e+00]] 4.891031774109361


In [4]:
table_policy = srp.MinSSRPFromPis(pi, S)
cyclic_policy = srp.StaticRP(np.array([0,1,2]))

[0.33333333 0.33333333 0.33333333]
[1. 1. 1.]
3
[1. 1. 1.]
[0.61803399 0.23606798 0.85410197]


In [5]:
simple_rule = drp.DPRP(lambdas, beta, S, drp.shortest_lengths)
simple_rule_time_diffs = drp.DPRP(lambdas, beta, S, drp.shortest_lengths, fully_observed = False)

In [6]:
#now simulate and compare
t = 2*60*60
xt1, wt1, _, _, _, _, _  = sym_sys.simulate(static_rand, S, t)
print(wt1[-1][1])
xt2, wt2, _, _, _, _, _  = sym_sys.simulate(static_markov, S, t)
print(wt2[-1][1])
xt3, wt3, _, _, _, _, _  = sym_sys.simulate(table_policy, S, t)
print(wt3[-1][1])
xt4, wt4, _, _, _, _, _  = sym_sys.simulate(cyclic_policy, S, t)
print(wt4[-1][1])
xt5, wt5, _, _, _, _, _  = sym_sys.simulate(simple_rule, S, t)
print(wt5[-1][1])
xt6, wt6, _, _, _, _, _  = sym_sys.simulate(simple_rule_time_diffs, S, t, fully_observed = False)
print(wt6[-1][1])

6.1285548840046635
4.857931773618174
4.900028127895295
4.881923991241851
3.8991426733479226
4.937150246312673


In [7]:
#now run VI to find a nice little DP program to help us out here

class DPVI:
    
    def __init__(self, psys, S, max_length = 100):
        self.n = ps.n
        self.psys = psys
        self.S = S
#         self.argmins = -1*np.ones((n, max_length, max_length, max_length))
        self.Q = -1*np.ones((n, max_length, max_length, max_length, n))
        
        
    def vi(self):
        pass